https://towardsdatascience.com/multi-class-metrics-made-simple-part-i-precision-and-recall-9250280bddc2

In [1]:
import numpy as np
import pandas as pd
import itertools as it
# from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import train_test_split
#from Orange.classification import tree
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
from sklearn import metrics
from sklearn.model_selection import GridSearchCV


from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
Data = pd.read_excel('Data.xlsx')
Data = Data.set_index('Date')

# Partir la data inicial en entrenamiento y validación para tomar submuestras del mismo conjunto siempre
Datatr = Data.reset_index()[Data.reset_index().index < np.floor(0.7*len(Data))].set_index('Date')
Datats = Data.reset_index()[Data.reset_index().index >= np.floor(0.7*len(Data))].set_index('Date')

#Ytr = Datatr['Class']
#Xtr = Datatr.drop('Class', axis = 1)

#Yts = Datats['Class']
#Xts = Datats.drop('Class', axis = 1)

In [3]:
#Hpyer_Parameters = [[1,2,3,4,5,6,7,8,9],[4,5,6]]
#set_parameters = [i for i in it.product(*Hpyer_Parameters)]

In [4]:
Epsilon = 0.2
delta = 0.1

### Parameters

In [5]:
param = Data.drop('Class',axis = 1).columns

In [6]:
h_aprox = len(param)
dimVC = h_aprox+1   # Poner aqui la dimVC

In [7]:
n_est = int(np.ceil((1/Epsilon)*(np.log(h_aprox) + np.log(1/delta))))
n_opt = int(np.ceil((1/Epsilon)*(np.log(dimVC) + np.log(1/delta))))
print('n_est',n_est,'n_opt',n_opt)

n_est 33 n_opt 33


In [8]:
#ns = [n_est+(n_opt-n_est+10*x) for x in range(1,9)]
ns = [n_est+int(((len(Datatr)-n_est)/(len(range(1,19)))*x)) for x in range(1,19)]

ns.insert(0,n_opt)
ns.insert(0,n_est)
print(ns)

[33, 33, 222, 412, 602, 792, 982, 1172, 1362, 1552, 1742, 1931, 2121, 2311, 2501, 2691, 2881, 3071, 3261, 3451]


In [9]:
clf = LogisticRegression()#.fit(X_train,y_train)
# Tuning
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values,scoring = 'accuracy')

In [10]:
grid_clf_acc

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.009, 0.01, 0.09, 1, 5, 10, 25],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [11]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
from sklearn import metrics
from sklearn.model_selection import GridSearchCV



acc=[]
ind = []
prec = []
rec = []
enes = []

for var in ns:
    print(var)
    #X_tr, X_ts, Y_tr, Y_ts = train_test_split(X, Y, train_size = var/X.shape[0], random_state=123)
    X_train = Datatr.sample(n=var, random_state=1).drop('Class', axis = 1)
    y_train = Datatr.sample(n=var, random_state=1)['Class']
    X_test = Datats.sample(n=int((var*0.3)/0.7), random_state=1).drop('Class', axis = 1)
    y_test = Datats.sample(n=int((var*0.3)/0.7), random_state=1)['Class']
    

    grid_clf_acc.fit(X_train, y_train)

    #reg = LinearRegression().fit(X_train, y_train)
    
    #clf = LogisticRegression().fit(X_train,y_train)
    
    y_pred = grid_clf_acc.predict(X_test)
    #y_pred = clf.predict(X_test)
    
    #print(y_pred)
    #print(reg.score(X, y))
    acc.append(accuracy_score(y_test,y_pred))
    

    c1 = [x for x in metrics.classification_report(y_test, y_pred, digits=3).split('\n')[2].split(' ') if x != '']
    c2 = [x for x in metrics.classification_report(y_test, y_pred, digits=3).split('\n')[3].split(' ') if x != '']
    c3 = [x for x in metrics.classification_report(y_test, y_pred, digits=3).split('\n')[4].split(' ') if x != '']
    micav=[x for x in metrics.classification_report(y_test,y_pred,digits=3).split('\n')[6].split(' ') if (x!='' and x!='avg')]

    enes.append(var)
    enes.append(var)
    enes.append(var)
    enes.append(var)
    
    ind.append(c1[0])
    ind.append(c2[0])
    ind.append(c3[0])
    ind.append(micav[0])
    
    prec.append(c1[1])
    prec.append(c2[1])
    prec.append(c3[1])
    prec.append(micav[1])
    
    rec.append(c1[2])
    rec.append(c2[2])
    rec.append(c3[2])
    rec.append(micav[2])

df = pd.DataFrame.from_dict({'muestras':enes,'ind':ind,'precision':prec,'recall':rec})
    
    
#reg.coef_
#reg.intercept_ 
#reg.predict(np.array([[3, 5]]))

33
33
222
412
602
792
982
1172
1362
1552
1742
1931
2121
2311
2501
2691
2881
3071
3261
3451


In [12]:
#import sklearn
sorted(list(sklearn.metrics.SCORERS.keys()))

NameError: name 'sklearn' is not defined

In [13]:
df.head(12)

,muestras,ind,precision,recall
0,33,-1,0.000,0.000
1,33,0,0.583,0.778
2,33,1,0.000,0.000
3,33,accuracy,0.500,14
4,33,-1,0.000,0.000
5,33,0,0.583,0.778
6,33,1,0.000,0.000
7,33,accuracy,0.500,14
8,222,-1,0.273,0.120
9,222,0,0.625,0.877


In [ ]:
#df[df.ind == '-1']['precision']

#iplot([{"x": df[df.ind == '-1'].sort_values('recall')['recall'], "y": df[df.ind == '-1'].sort_values('recall')['precision']},
#      {"x": df[df.ind == '0'].sort_values('recall')['recall'], "y": df[df.ind == '-1'].sort_values('recall')['precision']},
#      {"x": df[df.ind == '1'].sort_values('recall')['recall'], "y": df[df.ind == '-1'].sort_values('recall')['precision']}])

In [14]:
# Model Evaluation metrics 
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

Confusion Matrix : 
[[ 56 226  29]
 [ 51 853  25]
 [ 35 185  19]]


In [ ]:
#print(metrics.classification_report(y_test, y_pred, digits=3))

In [16]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

iplot([{"x": ns, "y": acc}])#,filename='acc', image='svg')

In [17]:
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

iplot([{"x": ns, "y": acc}])#,filename='acc', image='svg')